In [ ]:
import os, sys
project_dir = os.path.join(os.getcwd(),'..')
if project_dir not in sys.path:
    sys.path.append(project_dir)

from Sparse.modules.variational import LinearCD
from Sparse.modules.variational import VariationalLayer

In [ ]:
# Loss function 
from Sparse.modules.variational import VariationalLayer
from torch.nn.functional import cross_entropy
from torch import nn
class SGVBL(nn.Module):
    ''' 
        Stocastich Gradient Variational Bayes (SGVB) Loss function.
        More details in https://arxiv.org/pdf/1506.02557.pdf and https://arxiv.org/pdf/1312.6114.pdf
    '''

    def __init__(self, model, train_size, loss=cross_entropy):
        super(SGVBL, self).__init__()
        self.train_size = train_size
        self.net = model
        self.loss = loss

        self.variational_layers = []
        for module in model.modules():
            if isinstance(module, (VariationalLayer)):
                self.variational_layers.append(module)

    def forward(self, input, target, kl_weight=1.0):
        assert not target.requires_grad
        kl = 0.0
        for layer in self.variational_layers:
            kl += layer.kl_reg()
        
        # return self.loss(input, target) * self.train_size + kl_weight * kl    
        return self.loss(target, *input) * self.train_size + kl_weight * kl # Lo vi en concrete dropout que el kl_weight es 1/train_size

In [ ]:
class Model(nn.Module):
    def __init__(self, nb_features, weight_reg, drop_reg):
        super(Model, self).__init__()
        self.features = nn.Sequential(
            LinearCD(1, nb_features//2, bias=False, w_reg=weight_reg, drop_reg=drop_reg),
            nn.ReLU6(),
            LinearCD(nb_features//2, nb_features, w_reg=weight_reg, drop_reg=drop_reg),
            nn.ReLU6(),
            LinearCD(nb_features, nb_features, w_reg=weight_reg, drop_reg=drop_reg),
            nn.ReLU6(),
        )

        self.mu = LinearCD(nb_features, 1, w_reg=weight_reg, drop_reg=drop_reg)
        self.log_var = LinearCD(nb_features, 1, w_reg=weight_reg, drop_reg=drop_reg)

    def forward(self, x):
        h = self.features(x)
        mu = self.mu(h)
        log_var = self.log_var(h)
        return mu, log_var

def heteroscedastic_loss(true, mean, log_var):
    # Heteroscedatic, variation error is not constant during the samples.
    precision = torch.exp(-log_var)
    return torch.mean(torch.sum(precision * (true - mean)**2 + log_var, 1), 0)

# Original Experiment
... 

In [ ]:
import numpy as np
Ns = [10, 25, 50, 100, 1000, 10000] # Number of data points
Ns = np.array(Ns)
nb_epochs = [2000, 1000, 500, 200, 20, 2]
nb_val_size = 1000 # Validation size
nb_features = 1024 # Hidden layer size
Q = 1 # Data dimensionality
D = 1 # One mean, one log_var
K_test = 20 # Number of MC samples
nb_reps = 3 # Number of times to repeat experiment
batch_size = 64
l = 1e-6 # Lengthscale

In [ ]:
def gen_data(N):
    """
        Function to generate data
    """
    sigma = 1e0  # ground truth
    X = np.random.randn(N, Q)
    w = 2.
    b = 8.
    Y = X.dot(w) + b + sigma * np.random.randn(N, D)
    return X, Y

from matplotlib import pyplot as plt

plt.subplot(1,2,1)
X, Y = gen_data(10)
plt.scatter(X[:, 0], Y[:, 0], edgecolor='b')

plt.subplot(1,2,2)
X, Y = gen_data(10000)
plt.scatter(X[:, 0], Y[:, 0], edgecolor='b')
plt.xlim([-5, 5])
plt.ylim([-2, 20])
plt.show()

In [ ]:
import torch
from torch import optim
from torch.autograd import Variable

from tqdm import tqdm

def fit_model(nb_epoch, X, Y):
    N = X.shape[0]
    wr = l**2.
    dr = 2.
    model = Model(nb_features, wr, dr)
    model = model.cuda()
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    criterion = SGVBL(model, N, loss=heteroscedastic_loss)
    kl_weight = .6

    epoch_iterator = tqdm(
            range(nb_epoch),
            leave=True,
            unit="epoch",
            postfix={"tls": "%.4f" % 1},
        )

    for epoch in epoch_iterator:
        old_batch = 0
        kl_weight = min(kl_weight + 1e-2, 1.0)
        for batch in range(int(np.ceil(X.shape[0]/batch_size))):
            batch = (batch + 1)
            _x = X[old_batch: batch_size*batch]
            _y = Y[old_batch: batch_size*batch]
            
            x = Variable(torch.FloatTensor(_x)).cuda()
            y = Variable(torch.FloatTensor(_y)).cuda()
            
            output = model(x)
            loss = criterion(output, y, kl_weight)
             
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch % 10 == 0:
                epoch_iterator.set_postfix(tls="%.4f" % loss.detach().item())

        # print(loss.detach().item())
            
    return model


In [ ]:
N = Ns[4]
X, Y = gen_data(N + nb_val_size)
X_train, Y_train = X[:N], Y[:N]
X_val, Y_val = X[N:], Y[N:]
model = fit_model(50, X_train, Y_train)

In [ ]:
def plot2(X_train, Y_train, X_val, Y_val, means):
    indx = np.argsort(X_val[:, 0])
    _, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4,figsize=(12, 1.5), sharex=True, sharey=True)
    ax1.scatter(X_train[:, 0], Y_train[:, 0], c='y')
    ax1.set_title('Train set')
    ax2.plot(X_val[indx, 0], np.mean(means, 0)[indx], color='skyblue', lw=3)
    ax2.scatter(X_train[:, 0], Y_train[:, 0], c='y')
    ax2.set_title('+Predictive mean')
    for mean in means:
        ax3.scatter(X_val[:, 0], mean, c='b', alpha=0.2, lw=0)
    ax3.plot(X_val[indx, 0], np.mean(means, 0)[indx], color='skyblue', lw=3)
    ax3.set_title('+MC samples on validation X')
    ax4.scatter(X_val[:, 0], Y_val[:, 0], c='r', alpha=0.2, lw=0)
    ax4.set_title('Validation set')
    plt.show()
    

def logsumexp(a):
    a_max = a.max(axis=0)
    return np.log(np.sum(np.exp(a - a_max), axis=0)) + a_max

def test(Y_true, K_test, means, logvar):
    """
    Estimate predictive log likelihood:
    log p(y|x, D) = log int p(y|x, w) p(w|D) dw
                 ~= log int p(y|x, w) q(w) dw
                 ~= log 1/K sum p(y|x, w_k) with w_k sim q(w)
                  = LogSumExp log p(y|x, w_k) - log K
    :Y_true: a 2D array of size N x dim
    :MC_samples: a 3D array of size samples K x N x 2*D
    """
    k = K_test
    N = Y_true.shape[0]
    mean = means 
    logvar = logvar
    test_ll = -0.5 * np.exp(-logvar) * (mean - Y_val.squeeze())**2. - 0.5 * logvar - 0.5 * np.log(2 * np.pi) #Y_true[None]
    test_ll = np.sum(np.sum(test_ll, -1), -1)
    test_ll = logsumexp(test_ll) - np.log(k)
    pppp = test_ll / N  # per point predictive probability
    rmse = np.mean((np.mean(mean, 0) - Y_val.squeeze())**2.)**0.5
    return pppp, rmse


rep_results = []
model.train()
# model.eval()
MC_samples = [model(Variable(torch.FloatTensor(X_val)).cuda()) for _ in range(K_test)]
means = torch.stack([tup[0] for tup in MC_samples]).view(K_test, X_val.shape[0]).cpu().data.numpy()
logvar = torch.stack([tup[1] for tup in MC_samples]).view(K_test, X_val.shape[0]).cpu().data.numpy()
pppp, rmse = test(Y_val, K_test, means, logvar)
epistemic_uncertainty = np.var(means, 0).mean(0)
logvar = np.mean(logvar, 0)
aleatoric_uncertainty = np.exp(logvar).mean(0)
ps = np.array([torch.sigmoid(module.p_logit).cpu().data.numpy()[0] for module in model.modules() if hasattr(module, 'p_logit')])
plot(X_train, Y_train, X_val, Y_val, means)
rep_results += [(rmse, ps, aleatoric_uncertainty, epistemic_uncertainty)]

In [ ]:
print(torch.sigmoid(model.features[0].logit_p).mean())
print(torch.sigmoid(model.features[2].logit_p).mean())
print(torch.sigmoid(model.features[4].logit_p).mean())
print(torch.sigmoid(model.mu.logit_p).mean())
print(torch.sigmoid(model.log_var.logit_p).mean())

# Breast Cancer

In [ ]:
import os, sys
project_dir = os.path.join(os.getcwd(),'..')
if project_dir not in sys.path:
    sys.path.append(project_dir)

from Sparse.modules.variational import LinearCD
from Sparse.modules.variational.utils import SGVBL

In [ ]:
from sklearn import datasets
from torch.utils.data import Dataset, DataLoader
from torch.nn.functional import one_hot

class BreastCancer(Dataset):
    r'''
        Breast Cancer Wisconsin Dataset
    '''
    def __init__(self, normalize=False):
        dataset = datasets.load_breast_cancer()
        self.data = torch.tensor(dataset.data).float()
        self.targets = torch.tensor(dataset.target)
    
        if normalize:
            self.data /= torch.max(self.data, dim=0)[0]

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

    def __len__(self):
        return len(self.data)

In [ ]:
import torch
from torch import nn

class Model(nn.Module):
    def __init__(self, nb_features, weight_reg, drop_reg):
        super(Model, self).__init__()
        self.model = nn.Sequential(
            LinearCD(30, nb_features, w_reg=weight_reg, drop_reg=drop_reg),
            nn.ReLU(),
            LinearCD(nb_features, nb_features//2, w_reg=weight_reg, drop_reg=drop_reg),
            nn.ReLU(),
            LinearCD(nb_features//2, 2, w_reg=weight_reg, drop_reg=drop_reg)
        )
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

In [ ]:
dataset = BreastCancer(normalize=True)
loader = DataLoader(dataset, batch_size=64, shuffle=True)

In [ ]:
from tqdm import tqdm
from torch.nn.functional import cross_entropy

def train(model, dataset, batch_size = 64, n_epochs=10):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = SGVBL(model, len(dataset), cross_entropy)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    kl_weight = 1e-2

    epoch_iterator = tqdm(
            range(n_epochs),
            leave=True,
            unit="epoch",
            postfix={"tls": "%.4f" % 1},
        )

    for _ in epoch_iterator:
        kl_weight = min(kl_weight + 0.02, 1.0)
        for idx, (inputs, targets) in enumerate(loader):
            optimizer.zero_grad()

            inputs = inputs.to(device)
            targets = targets.to(device)
            pred = model(inputs)

            loss = criterion(pred, targets, kl_weight)
            loss.backward()
            optimizer.step()

            if idx % 10 == 0:
                epoch_iterator.set_postfix(tls="%.4f" % loss.item())

    return model

In [ ]:
l = 1e-2 # Lengthscale
w_reg = l**2 
d_reg = 1

model = Model(512, w_reg, d_reg)
model = train(model, dataset, n_epochs=300)

In [ ]:
(torch.sigmoid(model.model[0].logit_p) < .4).sum()

In [ ]:
loader = DataLoader(dataset, batch_size=64, shuffle=True)
x, y = next(iter(loader))
y

In [ ]:
mc_samples = 1000
samples = []
model.eval()
model.train()
with torch.no_grad():
    for i in range(mc_samples):
        samples.append(torch.softmax(model(x.cuda()), dim=1).cpu())

test = torch.stack(samples)

In [ ]:
from matplotlib import pyplot as plt 
samples_idx = 4
plt.subplot(1,2,1)
plt.plot(test[:, samples_idx, 0].detach().cpu().numpy(), label='0')
plt.subplot(1,2,1)
plt.plot(test[:, samples_idx, 1].detach().cpu().numpy(), label='1')
plt.title('Target: {}'.format(y[samples_idx]))
plt.legend()

In [ ]:
features_score, index = torch.sigmoid(model.model[0].logit_p).sort()

features_names = datasets.load_breast_cancer(as_frame=True).data.columns[index.cpu()]

print('Features:{}'.format(features_names))
print('Features Score:{}'.format(features_score))